In [4]:
import sys
sys.path.append("../../")
from src.utils import *
import censusdis.data as ced
alber_eq_us = "EPSG:5070"

In [5]:
import censusdis.data as ced

DATASET = "acs/acs5"
YEAR = 2020
CENSUS_VARS = {"NAME" : "NAME",
               "B01001_001E" : "total_pop",
               "B25003_001E": "total_households",
               "B08201_002E" : "households_no_vehicle",

}

us_state_pops = ced.download(DATASET, YEAR, CENSUS_VARS, state= "*", with_geometry=True).rename(CENSUS_VARS, axis = 1)
us_state_pops["vehicle_households"] = us_state_pops["total_households"] - us_state_pops["households_no_vehicle"]
us_state_pops.head(3)


,STATE,NAME,total_pop,total_households,households_no_vehicle,geometry,vehicle_households
0,42,Pennsylvania,12794885,5106601,544554,"POLYGON ((-80.51989 40.90666, -80.51963 40.911...",4562047
1,06,California,39346023,13103114,920362,"MULTIPOLYGON (((-118.60442 33.47855, -118.5987...",12182752
2,54,West Virginia,1807426,734235,62082,"POLYGON ((-82.6432 38.16909, -82.643 38.16956,...",672153


In [6]:
us_city_pops = gpd.read_file('../../data/city_pops_full.geojson')

In [7]:
us_cities_state = us_city_pops.merge(us_state_pops[["STATE", "NAME", "total_pop", "vehicle_households"]], left_on = "fip", right_on = "STATE")

In [8]:
us_cities_state["pop_prop"] = us_cities_state["interpolated_pop"] / us_cities_state["total_pop"]
us_cities_state["vehicle_household_prop"] = us_cities_state["interpolated_households_with_vehicle"] / us_cities_state["vehicle_households"]

In [9]:
city_trip_distance = pd.read_csv("../../data/city_trip_distances.csv")
state_trip_distance = pd.read_csv("../../data/state_trip_distances.csv")
state_trip_distance["STATE"] = state_trip_distance["STATE"].astype(str).str.zfill(2)
city_trip_distance.place_id = city_trip_distance.place_id.astype(str)

In [10]:
city_trip_distance = city_trip_distance.drop(columns=['Unnamed: 0'])
state_trip_distance = state_trip_distance.drop(columns=['Unnamed: 0'])


In [11]:
us_cities_state = us_cities_state.merge(city_trip_distance).merge(state_trip_distance)
us_cities_state["trip_distance_prop"] = us_cities_state["interpolated_tract_miles_traveled"] / us_cities_state["total_miles_state"]

In [12]:
city_weights = us_cities_state[["place_id", "NAME", "name", "state", "fip", "interpolated_pop", "pop_prop", "vehicle_household_prop", "trip_distance_prop"]]
city_weights.sample(3)

,place_id,NAME,name,state,fip,interpolated_pop,pop_prop,vehicle_household_prop,trip_distance_prop
335,231489518,Texas,Safran Electronics & Defense,Texas,48,17.470144,6.100882e-07,6.868199e-07,5.551516e-07
1126,297050654,Ohio,Madison,Ohio,39,3147.410827,2.695792e-04,2.920847e-04,2.276698e-04
1917,297106964,New Jersey,Cherry Hill Township,New Jersey,34,71390.462099,8.034564e-03,8.881326e-03,7.702554e-03


In [13]:
state_fuel = pd.read_csv("../../data/state_sales_23.csv")
state_fuel.head(3)
state_fuel = state_fuel.drop(columns="Unnamed: 0")

In [14]:
us_cities_state_fuel = state_fuel.merge(us_cities_state, left_on = "state", right_on = "state")
us_cities_state_fuel["fuel_weighted_pop"] = us_cities_state_fuel["total_fuel"] * us_cities_state_fuel["pop_prop"]
us_cities_state_fuel["fuel_weighted_vehicle_households"] = us_cities_state_fuel["total_fuel"] * us_cities_state_fuel["vehicle_household_prop"]
us_cities_state_fuel["fuel_weighted_trip_dist"] = us_cities_state_fuel["total_fuel"] * us_cities_state_fuel["trip_distance_prop"]

In [15]:
us_cities_state_fuel_wo_geom = us_cities_state_fuel.drop(columns='geometry')

In [16]:
us_cities_state_fuel_wo_geom.to_csv('../../data/fuel_sales_estimates_by_city.csv',index=False)